In this notebook we shall evaluate the quality of the obtained conformations from:
- BuildAMol
- GINGER
- FROG2

Our metrics will be:
- BuildAMol's counted clashes
- RDKit's UFF-based energy
- RMSD to UFF-minimized conformation

In [61]:
import buildamol as bam
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import pandas as pd
from pathlib import Path

BASE = Path(".").resolve()

First load the conformations

In [62]:
conformer_directories = BASE.glob("*_conformers")
conformer_datasets = {}
for _dir in conformer_directories:
    name = _dir.stem.replace("_conformers", "")
    files = list(_dir.glob("*.pdb"))
    mols = [bam.read_pdb(f) for f in files]

    for mol in mols:
        mol.remove_hydrogens()
        
    conformer_datasets[name] = mols

Regrettably, the ginger and frog2 data do not contain information on double bonds, so we have to add them manually to have all molecules actually represent the original (sucks big time...)

In [63]:
ginger = conformer_datasets["ginger"]
ginger_double_bonds = [
("O1", "C16"),
("O3", "C17"),
("C18", "C19"),
("C23", "C22"),
("C20", "C21"),
("C8", "C9"),
("C10", "C11"),
("C12", "C13"),
("C4", "C5"),
("C6", "C1"),
("C2", "C3"),
]
for mol in ginger:
    for bond in ginger_double_bonds:
        atom1 = mol.get_atom(bond[0])
        atom2 = mol.get_atom(bond[1])
        mol.set_bond_order(atom1, atom2, 2)

frog2 = conformer_datasets["frog2"]
frog2_double_bonds = [
("O3", "C3"),
("O1", "C2"),
("C6", "C5"),
("C7", "C8"),
("C4", "C9"),
("C12", "C17"),
("C16", "C15"),
("C13", "C14"),
("C18", "C19"),
("C20", "C21"),
("C22", "C23"),
]
for mol in frog2:
    for bond in frog2_double_bonds:
        atom1 = mol.get_atom(bond[0])
        atom2 = mol.get_atom(bond[1])
        mol.set_bond_order(atom1, atom2, 2)

Now we evaluate the metrics

In [64]:
clashes = {}
for dataset, mols in conformer_datasets.items():
    clashes[dataset] = [mol.count_clashes(coarse_precheck=False) for mol in mols]

Now we evaluate the molecular energy using RDKit

In [65]:
def energy(rdmol):
    return AllChem.UFFGetMoleculeForceField(rdmol).CalcEnergy()

energies = {}
for dataset, mols in conformer_datasets.items():
    energies[dataset] = [energy(mol.to_rdkit()) for mol in mols]

Now we also compute a UFF minimized version of the molecule using RDKIT

In [66]:
minimized = bam.read_pdb(BASE / "test_mol.pdb")
minimized.add_hydrogens()
before = minimized.copy()

minimized = minimized.to_rdkit()

AllChem.UFFOptimizeMolecule(minimized)

after = bam.molecule(minimized)
after.to_pdb("test_mol_minimized.pdb")

v = before.py3dmol()
v += after.py3dmol(color="lightgreen")
v.show()

print("Before:", energy(before.to_rdkit()))
print("After:", energy(minimized))

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Before: 1282.1597448196492
After: 59.76116215387487


Now compute the RMSD to the UFF minimized conformation

In [67]:
minimized = Chem.RemoveAllHs(minimized)

def rmsd(rdmol):
    try:
        score = AllChem.AlignMol(rdmol, minimized)
        return AllChem.GetBestRMS(rdmol, minimized)
    except Exception as e:
        return np.nan
    
rmsds = {}
for dataset, mols in conformer_datasets.items():
    rmsds[dataset] = [rmsd(mol.to_rdkit()) for mol in mols]

Now save the results

In [68]:
_clashes = {"dataset": [], "clashes": []}
for dataset, values in clashes.items():
    dataset = [dataset] * len(values)
    _clashes["dataset"].extend(dataset)
    _clashes["clashes"].extend(values)

_energies = {"dataset": [], "energy": []}
for dataset, values in energies.items():
    dataset = [dataset] * len(values)
    _energies["dataset"].extend(dataset)
    _energies["energy"].extend(values)

_rmsds = {"dataset": [], "rmsd": []}
for dataset, values in rmsds.items():
    dataset = [dataset] * len(values)
    _rmsds["dataset"].extend(dataset)
    _rmsds["rmsd"].extend(values)

_clashes = pd.DataFrame(_clashes)
_energies = pd.DataFrame(_energies)
_rmsds = pd.DataFrame(_rmsds)


_clashes.to_csv("clashes.csv", index=False)
_energies.to_csv("energies.csv", index=False)
_rmsds.to_csv("rmsds.csv", index=False)

In [69]:
# and also do a summary statistics
summary = {
    "dataset" : [],
    "mean_clashes" : [],
    "std_clashes" : [],
    "mean_energy" : [],
    "std_energy" : [],
    "mean_rmsd" : [],
    "std_rmsd" : [],
}
for dataset, values in clashes.items():
    summary["dataset"].append(dataset)
    summary["mean_clashes"].append(np.mean(values))
    summary["std_clashes"].append(np.std(values))

for dataset, values in energies.items():
    summary["mean_energy"].append(np.mean(values))
    summary["std_energy"].append(np.std(values))

for dataset, values in rmsds.items():
    summary["mean_rmsd"].append(np.nanmean(values))
    summary["std_rmsd"].append(np.nanstd(values))

summary = pd.DataFrame(summary)
summary.to_csv("summary.csv", index=False)